In [1]:
import regex as re
import pandas as pd
from src.utils.preprocessing import *

%load_ext autoreload
%autoreload 2

In [ ]:
to_ignore1 = ['name:  ___',
                'unit no:   ___',
                'admission date:  ___',
                'discharge date:   ___',
                'date of birth:  ___',
                'date of birth'
                'followup instructions:',
                'attending: ___.',
                'attending: ___',
                '___ no:   ___',
                'Unit ___:']

to_ignore2 = ['.',
                ')',
                '(',
                ':',
                ',',
                '/',
                '_']

to_ignore3 = ['primary care physician    found down',
              'date of birth',
              'no',
              'md',
              'tia',
              'date  birth',
              'sr',
              'admission date',
              'discharge date',
              'fr',
              'physical  physical exam',
              '#1 to',
              'lethargy']

to_inline = all_starts_and_ends

to_ignore1.extend(all_starts_and_ends)


def find_difference(string1, string2):
    for marker in to_inline:
        string1 = string1.lower().replace(marker.lower(), f'\n{marker.lower()}\n', 1)
        string2 = string2.lower().replace(marker.lower(), f'\n{marker.lower()}\n', 1)

    #print(string1)
    #print(string2)

    lines1 = [line.strip().lower() for line in string1.split('\n')]
    lines2 = [line.strip().lower() for line in string2.split('\n')]

    diff_lines = [line for line in lines1 if line not in lines2]
    diff_lines = [line for line in diff_lines if line not in string2]
    diff_string = '\n'.join(diff_lines)

    for ignore in to_ignore1:
        diff_string = diff_string.replace(ignore.lower(), '')
    
    for ignore in to_ignore2:    
        diff_string = diff_string.replace(ignore.lower(), '')

    for ignore in to_ignore3:
        diff_string = diff_string.replace(ignore.lower(), '')
    
    return diff_string.strip()

In [34]:
discharge_df = pd.read_csv('data/version1.4_filtered/train/discharge.csv.gz', compression='gzip')
target_df = pd.read_csv('data/version1.4_filtered/train/discharge_target.csv.gz', compression='gzip')

In [35]:
combined_df = build_combined_discharge(discharge_df, target_df)

In [5]:
final_df = extract_clean_sections_and_count_tokens(combined_df,section_to_next_section.keys())

Removing BHC and DI from the discharge


100%|██████████| 14719/14719 [00:05<00:00, 2533.44it/s]


Extracting features


100%|██████████| 14719/14719 [01:48<00:00, 135.23it/s]


Formating and cleaning sex section


...

100%|██████████| 14719/14719 [00:06<00:00, 2123.01it/s]


Formating and cleaning discharge_condition section


100%|██████████| 14719/14719 [00:00<00:00, 17170.56it/s]


Counting tokens in discharge_condition section


100%|██████████| 14719/14719 [00:10<00:00, 1420.11it/s]


In [12]:
display(final_df)

,hadm_id,text,discharge_instructions,brief_hospital_course,sex,sex_tokens,service,service_tokens,allergies,allergies_tokens,...,discharge_medications,discharge_medications_tokens,discharge_disposition,discharge_disposition_tokens,facility,facility_tokens,discharge_diagnosis,discharge_diagnosis_tokens,discharge_condition,discharge_condition_tokens
0,22595853,\nName: ___ Unit No: _...,"Dear Ms. ___,\nIt was a pleasure taking care o...","___ HCV cirrhosis c/b ascites, hiv on ART, h/o...",Sex:\nF\n,6,Service:\nMEDICINE\n,9,Allergies:\nNo Known Allergies / Adverse Drug ...,20,...,Discharge Medications:\nAlbuterol Inhaler 2 PU...,213,Discharge Disposition:\nHome\n,11,Facility:\nNone\n,7,Discharge Diagnosis:\nAscites from Portal HTN\n,18,Discharge Condition:\nMental Status: Clear and...,41
1,27897940,\nName: ___ Unit No: ___\n \...,"Mr. ___,\nYou were admitted after you fell and...",Mr. ___ is an ___ with history of AAA s/p repa...,Sex:\nM\n,6,Service:\nMEDICINE\n,9,Allergies:\nNo Known Allergies / Adverse Drug ...,20,...,Discharge Medications:\nClopidogrel 75 mg PO D...,246,Discharge Disposition:\nHome With Service\n,13,Facility:\nNone\n,7,Discharge Diagnosis:\nNasal fracture\nEpistaxi...,27,Discharge Condition:\nMental Status: Clear and...,41
2,24760295,\nName: ___ Unit No: ___\n...,It was a pleasure taking care of you during yo...,___ year old female with a history of CAD s/p ...,Sex:\nF\n,6,Service:\nMEDICINE\n,9,Allergies:\nlisinopril\n,11,...,Discharge Medications:\nAlbuterol Inhaler 2 PU...,545,Discharge Disposition:\nHome\n,11,Facility:\nNone\n,7,Discharge Diagnosis:\nPrimary:\nnon-ST elevati...,54,Discharge Condition:\nMental Status: Clear and...,41
3,21329021,\nName: ___ Unit No: ...,"Dear Mr. ___, \n \nWHY WERE YOU ADMITTED TO T...",SUMMARY: \n===================== \n___ male wi...,Sex:\nM\n,6,Service:\nMEDICINE\n,9,Allergies:\nNo Allergies/ADRs on File\n,17,...,Discharge Medications:\nAlbuterol Inhaler 2 PU...,516,Discharge Disposition:\nHome\n,11,Facility:\nNone\n,7,Discharge Diagnosis:\nPrimary Diagnosis: infla...,40,Discharge Condition:\nMental Status: Clear and...,41
4,25179393,\nName: ___. Unit No: ___\n \...,"Dear Mr. ___,\n\nThank you for choosing us for...",___ yo male with h/o Stage IV Hodgkin's lympho...,Sex:\nM\n,6,Service:\nMEDICINE\n,9,Allergies:\nRagweed / morphine / Percocet\n,19,...,Discharge Medications:\nAcyclovir 400 mg PO Q8...,284,Discharge Disposition:\nHome\n,11,Facility:\nNone\n,7,Discharge Diagnosis:\nNeutropenic Fever\n,18,Discharge Condition:\nMental Status: Clear and...,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14714,27119651,\nName: ___ Unit No: _...,"Dear Mr. ___,\n\nYou were admitted to the hosp...",___ year old man with Stage IV COPD CAD s/p CA...,Sex:\nM\n,6,Service:\nMEDICINE\n,9,Allergies:\nNo Known Allergies / Adverse Drug ...,20,...,Discharge Medications:\nHeparin 5000 UNIT SC T...,460,Discharge Disposition:\nExtended Care\n,13,Facility:\nNone\n,7,Discharge Diagnosis:\nEncephalopathy\nSacral d...,70,Discharge Condition:\nMental Status: Confused ...,46
14715,25548363,\nName: ___ Unit No: ___\...,"Dear Ms. ___,\n\nIt was a privilege taking car...",SUMMARY:\n===========\nMs. ___ is a ___ Female...,Sex:\nF\n,6,Service:\nMEDICINE\n,9,Allergies:\nNo Known Allergies / Adverse Drug ...,28,...,Discharge Medications:\nEnoxaparin Sodium 60 m...,259,Discharge Disposition:\nNone\n,11,Facility:\nNone\n,7,Discharge Diagnosis:\nHome\n \nDischarge Diagn...,66,Discharge Condition:\nMental Status: Clear and...,41
14716,24071062,\nName: ___ Unit No: ___\n ...,"Dear Ms. ___,\n\nYou have been hospitalized at...",___ h/o temporal lobe epilepsy p/w several gen...,Sex:\nF\n,6,Service:\nNEUROLOGY\n,10,Allergies:\nNo Known Allergies / Adverse Drug ...,20,...,Discharge Medications:\nVimpat 50 mg Tablet Si...,289,Discharge Disposition:\nHome\n,11,Facility:\nNone\n,7,Discharge Diagnosis:\nPrimary Diagnosis: Seizu...,19,Discharge Condition:\nMental Status: Clear and..

In [55]:
prompts_ = select_strategy(final_df, 'BHC', max_length=3000)

Selecting strategy: 100%|██████████| 14719/14719 [00:56<00:00, 260.00it/s]

Number of rows that exceed the maximum length: 0/14719


In [4]:
bhc_strategy = generate_strategies(bhc_importance_order, removeable_bhc)

for strat in bhc_strategy[:10]:
    print(strat)    

def has_duplicates(lst):
    seen = set()
    for sublist in lst:
        sublist_tuple = tuple(sublist)  # Convert sublist to tuple, as lists are unhashable
        if sublist_tuple in seen:
            return True
        seen.add(sublist_tuple)
    return False

print(len(bhc_strategy))
print(has_duplicates(bhc_strategy))

    

['sex', 'service', 'chief_complaint', 'history_of_present_illness', 'pertinent_results', 'physical_exam', 'major_surgical_procedures', 'past_medical_history', 'medication_on_admission', 'allergies', 'social_history', 'family_history']
['sex', 'service', 'chief_complaint', 'history_of_present_illness', 'pertinent_results', 'physical_exam', 'major_surgical_procedures', 'past_medical_history', 'medication_on_admission', 'allergies', 'social_history']
['sex', 'service', 'chief_complaint', 'history_of_present_illness', 'pertinent_results', 'physical_exam', 'major_surgical_procedures', 'past_medical_history', 'medication_on_admission', 'allergies', 'family_history']
['sex', 'service', 'chief_complaint', 'history_of_present_illness', 'pertinent_results', 'physical_exam', 'major_surgical_procedures', 'past_medical_history', 'medication_on_admission', 'social_history', 'family_history']
['sex', 'service', 'chief_complaint', 'history_of_present_illness', 'pertinent_results', 'physical_exam', 'ma

In [53]:
di_strategy = generate_strategies(di_importance_order, removeable_di)

print(len(di_strategy))
print(has_duplicates(di_strategy))


32
True


In [8]:
test_preprocessed = pd.read_json('data/inference_preprocessed/bhc_inference_input.jsonl', lines=True)

In [9]:
display(test_preprocessed)

,idx,prompt,reference
0,24962904,You are a medical assistant. Your task is to w...,Ms. ___ is a ___ female with history of \nCOPD...
1,22774359,You are a medical assistant. Your task is to w...,___ male with h/o Hodgkin's lymphoma C1D17 ABV...
2,29323205,You are a medical assistant. Your task is to w...,___ RH female with a PMHx of paramedian pontin...
3,20459702,You are a medical assistant. Your task is to w...,Hospitalization Summary: \nMs. ___ is an ___ y...
4,28755331,You are a medical assistant. Your task is to w...,___ with h/o psychosis admitted because of mul...
...,...,...,...
10957,23598426,You are a medical assistant. Your task is to w...,Right Abducens Nerve Palsy\nThe patient was ad...
10958,29933340,You are a medical assistant. Your task is to w...,Ms. ___ is a ___ y/o woman with a complicated ...
10959,22314636,You are a medical assistant. Your task is to w...,"___ PMH of Cirrhosis ___ hemochromatosis, c/b ..."
10960,20277361,You are a medical assistant. Your task is to w...,SUMMARY: \n___ y/o ___ immigrant with h/o alco...


In [10]:
print(test_preprocessed['prompt'][5])

You are a medical assistant. Your task is to write the brief hospital course corresponding to the following hospital discharge:

Sex:
M

Service:
MEDICINE

Allergies:
Penicillins / Tetracycline Analogues / Amoxicillin / IV Dye, 
Iodine Containing Contrast Media / Lactose

Chief Complaint:
Abnormal Stress Test, New AI

Major Surgical or Invasive Procedure:
None

History of Present Illness:
Mr. _ is a _ with a hx of CAD (s/p DES to LAD in 
_ 
who presented today for a routine stress echo (ordered by his 
PCP) and was found to have evidence of inducible ischemia and 
severe aortic insuficiency. The patient was in his usual state 
of health prior to the stress test. He retired from his 
_ job of _ years just 3 days ago. While on the 
treadmill the patient developed left shoulder pain (similar to 
previous anginal pains) and ST changes on EKG. Echocardiography 
revealed 4+ aortic regurgitation (patient was noted to be 1+ on 
last echo in _. After getting off the treadmill the pain 
went awa

In [9]:
test_preprocessed['nan_in'] = test_preprocessed['prompt'].progress_apply(lambda x: 'You are a medical assistant. Your task is to write the brief hospital course corresponding to the following hospital discharge:\n\nnan' in x)
filtered = test_preprocessed[test_preprocessed['nan_in'] == True]

100%|██████████| 10962/10962 [00:00<00:00, 109627.68it/s]


In [10]:
filtered['prompt'].describe()

count       0
unique      0
top       NaN
freq      NaN
Name: prompt, dtype: object

In [11]:
problems_df = combined_df.merge(filtered['idx'], left_on='hadm_id', right_on='idx', how='inner')

NameError: name 'combined_df' is not defined

In [ ]:
display(problems_df)

,hadm_id,text,discharge_instructions,brief_hospital_course,idx


In [ ]:
promblem_df_extracted = extract_clean_sections_and_count_tokens(problems_df, section_to_next_section.keys())

Removing BHC and DI from the discharge


1it [00:00, 38.39it/s]

Extracting features


ZeroDivisionError: integer division or modulo by zero

In [ ]:
display(promblem_df_extracted)

,hadm_id,text,discharge_instructions,brief_hospital_course,idx,sex,sex_tokens,service,service_tokens,allergies,...,discharge_medications,discharge_medications_tokens,discharge_disposition,discharge_disposition_tokens,facility,facility_tokens,discharge_diagnosis,discharge_diagnosis_tokens,discharge_condition,discharge_condition_tokens
0,23659790,\nName: ___ Unit No: ___\n...,"Dear Mr. ___,\n\nYou were admitted to the hosp...",Mr. ___ is a ___ year old man with recurrent D...,23659790,Sex:\nM\n,6,Service:\nMEDICINE\n,9,Allergies:\nProchlorperazine\n,...,Discharge Medications:\nAcyclovir 400 mg PO Q1...,303,Discharge Disposition:\nExtended Care\n,13,Facility:\nNone\n,7,Discharge Diagnosis:\nPrimary:\nDiffuse Large ...,81,Discharge Condition:\nMental Status: Confused ...,54
1,21531361,\nName: ___ Unit No: _...,"Dear Ms. ___,\nYou were admitted to ___ \nbeca...",BRIEF HOSPITAL COURSE\n===============\n___ ye...,21531361,Sex:\nF\n,6,Service:\nMEDICINE\n,9,Allergies:\nDemerol / Iodine / Augmentin / adh...,...,Discharge Medications:\nAspirin 325 mg PO DAIL...,561,Discharge Disposition:\nHome With Service\n,13,Facility:\nNone\n,7,Discharge Diagnosis:\nPrimary: CHF exacerbatio...,43,Discharge Condition:\nMental Status: Confused ...,49
2,21345164,\nName: ___. Unit No: __...,Dear ___ was a pleasure caring for you.\nYou w...,"___ w chronic pain, TBM, sarcoid, aspiration, ...",21345164,Sex:\nF\n,6,Service:\nMEDICINE\n,9,Allergies:\nKetorolac / Nalbuphine / Simvastat...,...,Discharge Medications:\nAcetaminophen 1000 mg ...,995,Discharge Disposition:\nExtended Care\n,13,Facility:\nNone\n,7,Discharge Diagnosis:\nPain after fall\nHyponat...,79,Discharge Condition:\nAmbulating with walker\n,16
3,28355415,\nName: ___ Unit No: ___\n \...,"Dear Mr ___,\n\nThank you for allowing us to p...","___ y/o man w/ h/o DM2, CAD s/p CABG ___, redo...",28355415,Sex:\nM\n,6,Service:\nMEDICINE\n,9,Allergies:\nIodine-Iodine Containing\n,...,Discharge Medications:\nAspirin 325 mg PO DAIL...,364,Discharge Disposition:\nExtended Care\n,13,Facility:\nNone\n,7,Discharge Diagnosis:\nMSSA Bacteremia\n,17,Discharge Condition:\nMental Status: Clear and...,60
4,20976134,\nName: ___ Unit No: ___\...,"Dear Mr. ___,\nIt was a pleasure to take care ...",Mr. ___ is a ___ year old man with h/o lung __...,20976134,Sex:\nM\n,6,Service:\nMEDICINE\n,9,Allergies:\nPenicillins / carboplatin\n,...,Discharge Medications:\nAtorvastatin 40 mg PO ...,160,Discharge Disposition:\nExtended Care\n,13,Facility:\nNone\n,7,Discharge Diagnosis:\nVertebral Fractures in C...,58,Discharge Condition:\nMental Status: Clear and...,52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1068,24176077,\nName: ___ Unit No: ___\n ...,"Dear Mr. ___,\n\nIt was a pleasure taking care...",___ with history of polysubstance abuse (repor...,24176077,Sex:\nM\n,6,Service:\nMEDICINE\n,9,Allergies:\nNo Known Allergies / Adverse Drug ...,...,Discharge Medications:\nAcetaminophen 650 mg P...,1019,Discharge Disposition:\nHome\n,11,Facility:\nNone\n,7,Discharge Diagnosis:\nPRIMARY DIAGNOSES\n\n \n...,62,Discharge Condition:\nMental Status: Clear and...,41
1069,22316403,\nName: ___ Unit No: ___...,"Dear Ms. ___,\n\nIt was a pleasure caring for ...","Ms. ___ is a ___ YO F with ILD, RA (on prednis...",22316403,Sex:\nF\n,6,Service:\nMEDICINE\n,9,Allergies:\nPPD black rubber mix\n,...,Discharge Medications:\nFoLIC Acid 1 mg PO DAI...,539,Discharge Disposition:\nHome\n,11,Facility:\nNone\n,7,Discharge Diagnosis:\nAcute issues:\n#Intersti...,58,Discharge Condition:\nMental Status: Clear and...,41
1070,26790899,\nName: ___ Unit No: ___\n...,"Dear Mr. ___,\n\nIt was our pleasure to care f...","___ dementia, CLL, atrial fibrillation that wa...",26790899,Sex:\nM\n,6,Service:\nMEDICINE\n,9,Allergies:\nNo Known Allergies / Adverse Drug ...,...,Discharge Medications:\ndigoxin 250 mcg Tablet...,415,Discharge Disposition:\nExtended Care\n,13,Facility:\nNone\n,7,Discharge Diagnosis:\nPrimary: C5 cervical

In [ ]:
promblem_df_extracted['final_input'] = promblem_df_extracted.progress_apply(lambda x: construct_final_input(x, section_to_next_section.keys(), section_to_next_section.keys()), axis=1)

100%|██████████| 1073/1073 [00:00<00:00, 14409.68it/s]


In [ ]:
display(print(promblem_df_extracted['final_input'][0]))

Sex:
M

Service:
MEDICINE

Allergies:
Prochlorperazine

Chief Complaint:
S/P fall at home

Major Surgical or Invasive Procedure:
Palliative radiation
Cyber Knife

History of Present Illness:
Mr. _ is a _ year old man with a history of HBV, HCV 
cirrhosis with gastric varices, portal hypertension and 
splenomegaly, as well as type I DM and recurrent DLBCL s/p CHOP, 
EPOCH and Rituxan who presented to the ER after sustaining a 
fall and landing on his left knee. He states that he was walking 
across his tile kitchen wearing his diabetic sneakers and 
tripped over his feet. He landed with his left knee on the tile. 
He was on the floor unable to get up for about 10 minutes. He 
denies dizziness, lightheadedness, chest pain, shortness of 
breath, loss of consciousness or head trauma. He remembers the 
entire event. He waited until his son was able to help him and 
came to the ER for pain and inability to ambulate. Of note, he 
denies recent memory changes, difficulty concentrating, changes

None

In [ ]:
promblem_df_extracted['brief_hospital_course_tokens'] = promblem_df_extracted['brief_hospital_course'].progress_apply(get_token_count)

promblem_df_extracted['final_input_tokens'] = promblem_df_extracted[[f"{section}_tokens" for section in section_to_next_section.keys()]].sum(axis=1) + len(section_to_next_section.keys())

display(promblem_df_extracted)

promblem_df_extracted['total_tokens']\
         = promblem_df_extracted['final_input_tokens'] \
            + promblem_df_extracted['brief_hospital_course_tokens']

display(promblem_df_extracted)

filtered_combined_discharges = promblem_df_extracted[promblem_df_extracted['total_tokens'] <= 5000]

100%|██████████| 1073/1073 [00:04<00:00, 218.03it/s]


,hadm_id,text,discharge_instructions,brief_hospital_course,idx,sex,sex_tokens,service,service_tokens,allergies,...,facility,facility_tokens,discharge_diagnosis,discharge_diagnosis_tokens,discharge_condition,discharge_condition_tokens,final_input,brief_hospital_course_tokens,final_input_tokens,total_tokens
0,23659790,\nName: ___ Unit No: ___\n...,"Dear Mr. ___,\n\nYou were admitted to the hosp...",Mr. ___ is a ___ year old man with recurrent D...,23659790,Sex:\nM\n,6,Service:\nMEDICINE\n,9,Allergies:\nProchlorperazine\n,...,Facility:\nNone\n,7,Discharge Diagnosis:\nPrimary:\nDiffuse Large ...,81,Discharge Condition:\nMental Status: Confused ...,54,Sex:\nM\n\nService:\nMEDICINE\n\nAllergies:\nP...,2144,5171,7315
1,21531361,\nName: ___ Unit No: _...,"Dear Ms. ___,\nYou were admitted to ___ \nbeca...",BRIEF HOSPITAL COURSE\n===============\n___ ye...,21531361,Sex:\nF\n,6,Service:\nMEDICINE\n,9,Allergies:\nDemerol / Iodine / Augmentin / adh...,...,Facility:\nNone\n,7,Discharge Diagnosis:\nPrimary: CHF exacerbatio...,43,Discharge Condition:\nMental Status: Confused ...,49,Sex:\nF\n\nService:\nMEDICINE\n\nAllergies:\nD...,1457,5639,7096
2,21345164,\nName: ___. Unit No: __...,Dear ___ was a pleasure caring for you.\nYou w...,"___ w chronic pain, TBM, sarcoid, aspiration, ...",21345164,Sex:\nF\n,6,Service:\nMEDICINE\n,9,Allergies:\nKetorolac / Nalbuphine / Simvastat...,...,Facility:\nNone\n,7,Discharge Diagnosis:\nPain after fall\nHyponat...,79,Discharge Condition:\nAmbulating with walker\n,16,Sex:\nF\n\nService:\nMEDICINE\n\nAllergies:\nK...,2899,6762,9661
3,28355415,\nName: ___ Unit No: ___\n \...,"Dear Mr ___,\n\nThank you for allowing us to p...","___ y/o man w/ h/o DM2, CAD s/p CABG ___, redo...",28355415,Sex:\nM\n,6,Service:\nMEDICINE\n,9,Allergies:\nIodine-Iodine Containing\n,...,Facility:\nNone\n,7,Discharge Diagnosis:\nMSSA Bacteremia\n,17,Discharge Condition:\nMental Status: Clear and...,60,Sex:\nM\n\nService:\nMEDICINE\n\nAllergies:\nI...,2101,4895,6996
4,20976134,\nName: ___ Unit No: ___\...,"Dear Mr. ___,\nIt was a pleasure to take care ...",Mr. ___ is a ___ year old man with h/o lung __...,20976134,Sex:\nM\n,6,Service:\nMEDICINE\n,9,Allergies:\nPenicillins / carboplatin\n,...,Facility:\nNone\n,7,Discharge Diagnosis:\nVertebral Fractures in C...,58,Discharge Condition:\nMental Status: Clear and...,52,Sex:\nM\n\nService:\nMEDICINE\n\nAllergies:\nP...,1342,5420,6762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1068,24176077,\nName: ___ Unit No: ___\n ...,"Dear Mr. ___,\n\nIt was a pleasure taking care...",___ with history of polysubstance abuse (repor...,24176077,Sex:\nM\n,6,Service:\nMEDICINE\n,9,Allergies:\nNo Known Allergies / Adverse Drug ...,...,Facility:\nNone\n,7,Discharge Diagnosis:\nPRIMARY DIAGNOSES\n\n \n...,62,Discharge Condition:\nMental Status: Clear and...,41,Sex:\nM\n\nService:\nMEDICINE\n\nAllergies:\nN...,1323,6324,7647
1069,22316403,\nName: ___ Unit No: ___...,"Dear Ms. ___,\n\nIt was a pleasure caring for ...","Ms. ___ is a ___ YO F with ILD, RA (on prednis...",22316403,Sex:\nF\n,6,Service:\nMEDICINE\n,9,Allergies:\nPPD black rubber mix\n,...,Facility:\nNone\n,7,Discharge Diagnosis:\nAcute issues:\n#Intersti...,58,Discharge Condition:\nMental Status: Clear and...,41,Sex:\nF\n\nService:\nMEDICINE\n\nAllergies:\nP...,1360,5970,7330
1070,26790899,\nName: ___ Unit No: ___\n...,"Dear Mr. ___,\n\nIt was our pleasure to care f...","___ dementia, CLL, atrial fibrillation that wa...",26790899,Sex:\nM\n,6,Service:\nMEDICINE\n,9,Allergies:\nNo Known Allergies / Adverse Drug ...,...,Facility:\nNone\n,7,Discharge Diagnosis:\nPrimary: C5 cervical fra...,82,Discharge Condition:\nMental Status: Confused ...,51,Sex:\nM\n\nService:\nMEDICINE\n\nAllergies:\nN...,2062,5681,7743
1071,28875588,\nName: ___ Unit No: ___\n...,"Dear Mr. ___,\n\nIt was a pleasure taking care...","___ h/o HIV (CD4 306 on ___, VL 200K ___ on AR...",28875588,Sex:\nM\n,6,Service:\nMEDICINE\n,9,Allergies:\nphen

,hadm_id,text,discharge_instructions,brief_hospital_course,idx,sex,sex_tokens,service,service_tokens,allergies,...,facility,facility_tokens,discharge_diagnosis,discharge_diagnosis_tokens,discharge_condition,discharge_condition_tokens,final_input,brief_hospital_course_tokens,final_input_tokens,total_tokens
0,23659790,\nName: ___ Unit No: ___\n...,"Dear Mr. ___,\n\nYou were admitted to the hosp...",Mr. ___ is a ___ year old man with recurrent D...,23659790,Sex:\nM\n,6,Service:\nMEDICINE\n,9,Allergies:\nProchlorperazine\n,...,Facility:\nNone\n,7,Discharge Diagnosis:\nPrimary:\nDiffuse Large ...,81,Discharge Condition:\nMental Status: Confused ...,54,Sex:\nM\n\nService:\nMEDICINE\n\nAllergies:\nP...,2144,5171,7315
1,21531361,\nName: ___ Unit No: _...,"Dear Ms. ___,\nYou were admitted to ___ \nbeca...",BRIEF HOSPITAL COURSE\n===============\n___ ye...,21531361,Sex:\nF\n,6,Service:\nMEDICINE\n,9,Allergies:\nDemerol / Iodine / Augmentin / adh...,...,Facility:\nNone\n,7,Discharge Diagnosis:\nPrimary: CHF exacerbatio...,43,Discharge Condition:\nMental Status: Confused ...,49,Sex:\nF\n\nService:\nMEDICINE\n\nAllergies:\nD...,1457,5639,7096
2,21345164,\nName: ___. Unit No: __...,Dear ___ was a pleasure caring for you.\nYou w...,"___ w chronic pain, TBM, sarcoid, aspiration, ...",21345164,Sex:\nF\n,6,Service:\nMEDICINE\n,9,Allergies:\nKetorolac / Nalbuphine / Simvastat...,...,Facility:\nNone\n,7,Discharge Diagnosis:\nPain after fall\nHyponat...,79,Discharge Condition:\nAmbulating with walker\n,16,Sex:\nF\n\nService:\nMEDICINE\n\nAllergies:\nK...,2899,6762,9661
3,28355415,\nName: ___ Unit No: ___\n \...,"Dear Mr ___,\n\nThank you for allowing us to p...","___ y/o man w/ h/o DM2, CAD s/p CABG ___, redo...",28355415,Sex:\nM\n,6,Service:\nMEDICINE\n,9,Allergies:\nIodine-Iodine Containing\n,...,Facility:\nNone\n,7,Discharge Diagnosis:\nMSSA Bacteremia\n,17,Discharge Condition:\nMental Status: Clear and...,60,Sex:\nM\n\nService:\nMEDICINE\n\nAllergies:\nI...,2101,4895,6996
4,20976134,\nName: ___ Unit No: ___\...,"Dear Mr. ___,\nIt was a pleasure to take care ...",Mr. ___ is a ___ year old man with h/o lung __...,20976134,Sex:\nM\n,6,Service:\nMEDICINE\n,9,Allergies:\nPenicillins / carboplatin\n,...,Facility:\nNone\n,7,Discharge Diagnosis:\nVertebral Fractures in C...,58,Discharge Condition:\nMental Status: Clear and...,52,Sex:\nM\n\nService:\nMEDICINE\n\nAllergies:\nP...,1342,5420,6762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1068,24176077,\nName: ___ Unit No: ___\n ...,"Dear Mr. ___,\n\nIt was a pleasure taking care...",___ with history of polysubstance abuse (repor...,24176077,Sex:\nM\n,6,Service:\nMEDICINE\n,9,Allergies:\nNo Known Allergies / Adverse Drug ...,...,Facility:\nNone\n,7,Discharge Diagnosis:\nPRIMARY DIAGNOSES\n\n \n...,62,Discharge Condition:\nMental Status: Clear and...,41,Sex:\nM\n\nService:\nMEDICINE\n\nAllergies:\nN...,1323,6324,7647
1069,22316403,\nName: ___ Unit No: ___...,"Dear Ms. ___,\n\nIt was a pleasure caring for ...","Ms. ___ is a ___ YO F with ILD, RA (on prednis...",22316403,Sex:\nF\n,6,Service:\nMEDICINE\n,9,Allergies:\nPPD black rubber mix\n,...,Facility:\nNone\n,7,Discharge Diagnosis:\nAcute issues:\n#Intersti...,58,Discharge Condition:\nMental Status: Clear and...,41,Sex:\nF\n\nService:\nMEDICINE\n\nAllergies:\nP...,1360,5970,7330
1070,26790899,\nName: ___ Unit No: ___\n...,"Dear Mr. ___,\n\nIt was our pleasure to care f...","___ dementia, CLL, atrial fibrillation that wa...",26790899,Sex:\nM\n,6,Service:\nMEDICINE\n,9,Allergies:\nNo Known Allergies / Adverse Drug ...,...,Facility:\nNone\n,7,Discharge Diagnosis:\nPrimary: C5 cervical fra...,82,Discharge Condition:\nMental Status: Confused ...,51,Sex:\nM\n\nService:\nMEDICINE\n\nAllergies:\nN...,2062,5681,7743
1071,28875588,\nName: ___ Unit No: ___\n...,"Dear Mr. ___,\n\nIt was a pleasure taking care...","___ h/o HIV (CD4 306 on ___, VL 200K ___ on AR...",28875588,Sex:\nM\n,6,Service:\nMEDICINE\n,9,Allergies:\nphen

In [ ]:
strat_df = pd.read_json('data/inference_preprocessed/bhc_inference_input.jsonl', lines=True)

,hadm_id,text,discharge_instructions,brief_hospital_course,idx,sex,sex_tokens,service,service_tokens,allergies,...,facility,facility_tokens,discharge_diagnosis,discharge_diagnosis_tokens,discharge_condition,discharge_condition_tokens,final_input,brief_hospital_course_tokens,final_input_tokens,total_tokens
